In [1]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pandas as pd
from nsepy import get_history
from docplex.mp.model import Model
from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.optimization import QuadraticProgram
from qiskit_finance.data_providers import RandomDataProvider
from qiskit.optimization.converters import LinearEqualityToPenalty
import pennylane as qml
from qiskit.optimization.applications.ising.common import sample_most_likely

In [2]:
num_assets = 3
# Generate expected return and covariance matrix from (random) time-series
stocks = [("TICKER%s" % i) for i in range(num_assets)]
data = RandomDataProvider(tickers=stocks,
                 start=datetime.datetime(2016,1,1),
                 end=datetime.datetime(2016,1,30))
data.run()

In [3]:
#Calculate mean returns
mu = data.get_period_return_mean_vector() 
 
#Calculate mean covariance
sigma = data.get_period_return_covariance_matrix()
q = 0.5                   # set risk factor
budget = 2          # set budget (B)
penalty = num_assets      # set parameter to scale the budget penalty term
mdl = Model('docplex model')
x = mdl.binary_var_list(num_assets)
# set objective function: 
#
#     maximize { mu^T * x - q * x^T * sigma * x }
#
objective = mdl.sum([mu[i] * x[i] for i in range(num_assets)])  
# mu^T * x
objective -= q * mdl.sum([sigma[i][j]*x[i]*x[j] for i in range(num_assets) for j in range(num_assets)])
mdl.maximize(objective)
# add budget constraint: 
#
#     1^T * x == budget
#
cost = mdl.sum([x[i] for i in range(num_assets)])
mdl.add_constraint(cost == budget, ctname='budget')
# converting to Quadratic Program
mod = QuadraticProgram()        
mod.from_docplex(mdl)
#removing the constraint to create the QUBO
lineq2penalty = LinearEqualityToPenalty(penalty) 
qubo = lineq2penalty.convert(mod)
#converting QUBO to an Ising Hamiltonian
H, offset = qubo.to_ising()
H = H.to_legacy_op()  #coverting it to a legacy operator
H.print_details()

'IIZ\t(1.4890162084841014+0j)\nIZI\t(1.4990044628210766+0j)\nZII\t(1.5010844950763147+0j)\nIZZ\t(1.4999517804084235+0j)\nZIZ\t(1.4998832592835347+0j)\nZZI\t(1.4999977567411256+0j)\n'

In [4]:
def pauli_operator_to_dict(pauli_operator):
    
    d = pauli_operator.to_dict()
    paulis = d['paulis']
    paulis_dict = {}
    for x in paulis:
        label = x['label']
        coeff = x['coeff']['real']
        paulis_dict[label] = coeff
    return paulis_dict
pauli_dict = pauli_operator_to_dict(H)
print(pauli_dict)

{'IIZ': 1.4890162084841014, 'IZI': 1.4990044628210766, 'ZII': 1.5010844950763147, 'IZZ': 1.4999517804084235, 'ZIZ': 1.4998832592835347, 'ZZI': 1.4999977567411256}


In [5]:
def ansatz(theta):
    
    qml.RX(theta[0], wires=0)
    qml.RX(theta[1], wires=1)
    qml.RX(theta[2], wires=2)
    qml.CNOT(wires=[0,1])
    qml.CNOT(wires=[0,2])
    qml.CNOT(wires=[1,2])
    qml.RX(theta[3], wires=0)
    qml.RX(theta[4], wires=1)
    qml.RX(theta[5], wires=2)
    qml.CNOT(wires=[0,1])
    qml.CNOT(wires=[0,2])
    qml.CNOT(wires=[1,2])
    qml.RX(theta[6], wires=0)
    qml.RX(theta[7], wires=1)
    qml.RX(theta[8], wires=2)
    
    return ansatz

In [6]:
#qml.qnode(dev1)
def circuit_IIZ(params):
    ansatz(params)
    return qml.expval(qml.Identity(0)@qml.Identity(1)@qml.PauliZ(2))
#qml.qnode(dev1)
def circuit_IIZ(params):
    ansatz(params)
    return qml.expval(qml.Identity(0)@qml.Identity(1)@qml.PauliZ(2))
#qml.qnode(dev1)
def circuit_IZI(params):
    ansatz(params)
    return qml.expval(qml.Identity(0)@qml.PauliZ(1)@qml.Identity(2))
#qml.qnode(dev1)
def circuit_ZII(params):
    ansatz(params)
    return qml.expval(qml.PauliZ(0)@qml.Identity(1)@qml.Identity(2))
#qml.qnode(dev1)
def circuit_IZZ(params):
    ansatz(params)
    return qml.expval(qml.Identity(0)@qml.PauliZ(1)@qml.PauliZ(2))
#qml.qnode(dev1)
def circuit_ZIZ(params):
    ansatz(params)
    return qml.expval(qml.PauliZ(0)@qml.Identity(1)@qml.PauliZ(2))
#qml.qnode(dev1)
def circuit_ZZI(params):
    ansatz(params)
    return qml.expval(qml.PauliZ(0)@qml.PauliZ(1)@qml.Identity(2))
#qml.qnode(dev1)
def circuit_IZI(params):
    ansatz(params)
    return qml.expval(qml.Identity(0)@qml.PauliZ(1)@qml.Identity(2))
#qml.qnode(dev1)
def circuit_ZII(params):
    ansatz(params)
    return qml.expval(qml.PauliZ(0)@qml.Identity(1)@qml.Identity(2))
#qml.qnode(dev1)
def circuit_IZZ(params):
    ansatz(params)
    return qml.expval(qml.Identity(0)@qml.PauliZ(1)@qml.PauliZ(2))
#qml.qnode(dev1)
def circuit_ZIZ(params):
    ansatz(params)
    return qml.expval(qml.PauliZ(0)@qml.Identity(1)@qml.PauliZ(2))
#qml.qnode(dev1)
def circuit_ZZI(params):
    ansatz(params)
    return qml.expval(qml.PauliZ(0)@qml.PauliZ(1)@qml.Identity(2))

In [7]:
def vqe(parameters):
    
    expval_ZII = pauli_dict['ZII'] * circuit_ZII(parameters)  
    expval_IIZ = pauli_dict['IIZ'] * circuit_IIZ(parameters)
    expval_IZI = pauli_dict['IZI'] * circuit_IZI(parameters)
    expval_ZZI = pauli_dict['ZZI'] * circuit_ZZI(parameters)
    expval_ZIZ = pauli_dict['ZIZ'] * circuit_ZIZ(parameters)
    expval_IZZ = pauli_dict['IZZ'] * circuit_IZZ(parameters)
    
    # summing the measurement results
    total_expval= expval_ZII + expval_IIZ + expval_ZIZ +  expval_ZZI + expval_ZIZ + expval_IZZ
    
    return total_expval

In [8]:
opt = qml.GradientDescentOptimizer(stepsize=0.5)
value = []
    # optimize parameters in objective
params = np.random.rand(9)
steps = 500
for i in range(steps):
    params = opt.step(vqe, params)
    value.append(vqe(params))
plt.plot(value)

/Users/vaidehi/opt/anaconda3/lib/python3.9/site-packages/pennylane/_grad.py:105: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


TypeError: unsupported operand type(s) for *: 'float' and 'MeasurementProcess'

In [9]:
#qml.qnode(dev1)
def final_circ(params):
    ansatz(params)
return qml.probs(wires=[0,1,2])

SyntaxError: 'return' outside function (2175665475.py, line 4)

In [10]:
plt.bar(['000', '001', '010', '011', '100', '101', '110', '111'], final_circ(params[0]))

IndexError: invalid index to scalar variable.

In [11]:
exact_eigensolver = NumPyMinimumEigensolver(H)
result = exact_eigensolver.run()
sample_most_likely(result.eigenstate)

#exact_eigen = NumPyMinimumEigensolver()
#exact_eigensolver = MinimumEigenOptimizer(exact_eigen)
#result = exact_eigensolver.solve()
#sample_most_likely(result.eigenstate)




AttributeError: 'NumPyMinimumEigensolver' object has no attribute 'run'